# Actividad Support Vector Machine
En esta actividad, repetiremos el análisis para detección de cancer, usando esta vez SVM

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

## Data

In [ ]:
from sklearn.datasets import load_breast_cancer
loadData = load_breast_cancer(as_frame=True)
data = loadData['data']
data.columns = data.columns.str.replace(' ','_')
target = loadData['target']
target = 1-target # para que "1" sea "Maligno"

OJO: este método necesita escalar los datos

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataNorm = scaler.fit_transform(data)

Separamos los datos en un set de entrenamiento (`X_train` y `y_train`) para calibrar el modelo, y un set de test (`X_test`, `y_test`) para evaluarlo.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataNorm, target, random_state=0)



## Parte 1:  Entrenar un modelo SVM y evaluar su performance.

Genere un modelo de SVM y calibrelo para los datos de entrenamiento.  Con esto haga una predicción para los datos de `X_test`, y guarde estos resultados en la variable `prediccion`

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(<Datos entrenamiento>, <Target entrenamiento>)
prediccion=NB.predict(<Datos test>)

Evalue el performance de su modelo, para las distintas métricas. Para esto, recuerde comparar los resultados reales `y_test` con los resultados predecidos `prediccion`.

Recuerde que puede usar distintas herramientas, por ejemplo:
- Accuracy : `mt.accuracy_score(<reales>,<prediccion>)` 
- Precision: `mt.precision_score(<reales>,<prediccion>)` 
- Recall: `mt.recall_score(<reales>,<prediccion>)` 
- F1: `mt.f1_score(<reales>,<prediccion>)` 
- Reporte general: `mt.classification_report(<reales>,<prediccion>)`
- Matriz confusión: `mt.confusion_matrix(<reales>,<prediccion>)`
- Matriz confusión (plot): `mt.ConfusionMatrixDisplay(mt.confusion_matrix(<reales>,<prediccion>)).plot()`

Recuerde F1= 2TP/(2TP+FN+FP), recall= TP/(TP+FP) y precision= TP/(TP+FN)

In [ ]:
from sklearn import metrics as mt
# Escriba aquí sus evaluaciones





## Parte 2 : Repetir la evaluación usando cross-validation
Para major certeza de lo obtenido, haremos un `KFold` (sugerencia: usar 10-fold), y usaremos `cross_validate` para evaluar cada una de las métricas, reportando su media y desviación estandar.  Para esto usamos los datos completos (o sea, no `X_train`, ...) ya que `cross_validate` se encargará de separar el conjunto train y test.

In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics as mt
from sklearn.model_selection import cross_validate


#Creamos un objeto de la clase KFold 
kf = KFold(n_skf = KFold(n_splits= <NUMERO DE FOLDS ELEGIDO> )

    
#Cree un objeto de la clase SVC. No necesita hacer .fit, cross_validate se encarga de eso
svm = SVC()



#Hacemos cross_validate para las distintas métricas. 
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(svm, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=metrica, cv=kf, return_train_score=True)    
    print ('Score',metrica,'train:', res['train_score'].mean(),'+-',res['train_score'].std())
    print ('Score',metrica,'test:', res['test_score'].mean(),'+-',res['test_score'].std())





## Parte 3 : Calibrar el modelo para distintos kernel y penalizadores de arbol usando cross-validation
Evaluamos nuevamente el modelo ajustando los tipos de kernel, y el factor `C` que penaliza los datos que violan la banda separadora del modelo.

Este modelo requiere mas recursos computacionales, por lo que escoja una metrica (`'accuracy'`, `'recall'`, `'precision'`, `'f1'`) que utilizará para escoger el mejor.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

#Generando los k-fold
kf = KFold(n_splits=10)

# Escoja la métrica que usará
metrica = <SU METRICA> # por ejemplo, metrica='accuracy'

# guardando los resultados
res = pd.DataFrame(columns=['kernel','C','tipo','mean','std'])
# ciclo para distintos tipos y penalizadores
for kernelTipo in ['linear', 'poly', 'rbf', 'sigmoid']:
    for valC in [0.1, 1.0, 10]: ## Modifique esta lista si lo desea para probar otros valores
        svm = SVC(kernel=kernelTipo, C=valC)
        results = cross_validate(svm, dataNorm,target, cv=kf, return_train_score=True, scoring=metrica)
        res.loc[len(res)] = [kernelTipo,valC,'train',results['train_score'].mean(),results['train_score'].std()]
        res.loc[len(res)] = [kernelTipo,valC,'test',results['test_score'].mean(),results['test_score'].std()]







Si todo resultó bien, grafique los resultados con el siguiente código:

In [ ]:
(ggplot(res)+
 aes(x="C",y="mean",ymin="mean-std",ymax="mean+std", group='tipo',color='tipo')+
 geom_point()+
 geom_errorbar()+ 
 labs(x="C",y=metrica)+ scale_x_log10()+ 
 theme_bw() + facet_wrap('kernel')
)



**Pregunta**: ¿Cuál es el mejor `kernel` y penalizador `C` a su parecer?  Escoga una combinación como "la mejor"

## Parte 4: Comparación con los métodos anteriores
Comparemos el performance obtenido para las distintas métricas, con los modelos anteriormente ajustados.  Repita el comando, usando `SVC` para los parámetros escogidos

In [ ]:

#Creando el objeto con sus características 
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier



scaler = StandardScaler()
dataNorm = scaler.fit_transform(data)

results = pd.DataFrame(columns=['Metodo','Tipo', 'Metrica', 'ScoreMean', 'ScoreStd'])

KNNmodel = KNeighborsClassifier(n_neighbors=<SU NUMERO DE VECINOS PARA KNN>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(KNNmodel, dataNorm, target, cv=kf, return_train_score=True, scoring=metrica)    
    results.loc[len(results)] = ['KNN','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['KNN','test',metrica,res['test_score'].mean(),res['test_score'].std()]

NB = GaussianNB()
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(NB, data,target, cv=kf, return_train_score=True, scoring=metrica)
    results.loc[len(results)] = ['NB','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['NB','test',metrica,res['test_score'].mean(),res['test_score'].std()]

AD = DecisionTreeClassifier(max_depth=<PROFUNDIDAD_ESCOGIDA>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(AD, data,target, cv=kf, return_train_score=True, scoring=metrica)
    results.loc[len(results)] = ['AD','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['AD','test',metrica,res['test_score'].mean(),res['test_score'].std()]

print('SVM')
svm = SVC(kernel=<SU KERNEL ESCOGIDO>, C=<SU PENALIZADOR ESCOGIDO>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(svm, dataNorm,target, cv=kf, return_train_score=True, scoring=metrica)
    results.loc[len(results)] = ['SVM','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['SVM','test',metrica,res['test_score'].mean(),res['test_score'].std()]







In [ ]:
(ggplot(results)+
 aes(x='Metodo',y="ScoreMean",ymin="ScoreMean-ScoreStd",ymax="ScoreMean+ScoreStd",color='Tipo')+
 geom_point()+ geom_errorbar()+ 
 labs(y="Score")+
 facet_wrap('Metrica')+
 theme_bw()
)







**Pregunta**: ¿Con cuál método se quedaría?
    